<a href="https://colab.research.google.com/github/victorknox/RedditAnalysis/blob/main/Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Correlation between Emotions and Moral Sentiments

In [163]:
# Importing the Reddit Comments dataset
import pandas as pd
data = pd.read_csv('Reddit_comments.csv', names=['title', 'body', 'votes', 'subreddit name', 'time', 'date', 'author'])
print(data.head())

                   title  ...         author
0                  title  ...         author
1  Twice - Momo deepfake  ...  beef_smuggler
2  Twice - Momo Deepfake  ...        zamin13
3  Twice - Momo Deepfake  ...       CodaAyax
4  Twice - Momo Deepfake  ...         Emoome

[5 rows x 7 columns]


In [164]:
# Importing the Moral Foundations Dictionary
import pandas as pd
MFD = pd.read_csv('MFD.csv')
print(MFD.head())

         Word  tag1  tag2  tag3
0        safe     1   NaN   NaN
1       peace     1   NaN   NaN
2  compassion     1   NaN   NaN
3      empath     1   NaN   NaN
4     sympath     1   NaN   NaN


In [165]:
comments = data['body']
posts = data['title'].unique()

print(len(comments))
print(len(posts))

86325
6834


In [166]:
pip install text2emotion

## Functions used

In [168]:
import pandas as pd

# Pearson coefficient
def pcorr(a, b):
  x = pd.Series(a)
  y = pd.Series(b)
  return x.corr(y)

In [169]:
import pandas as pd

# Spearman coefficient
def scorr(a, b):
  x = pd.Series(a)
  y = pd.Series(b)
  return x.corr(y, method='spearman')

In [170]:
def get_morality(all_sentiments, post, MFD):
  moral_sentiments_total = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  moral_sentiments = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  for i in range(len(MFD['Word'])):
    if MFD.iloc[i].Word in str(post):
      if MFD.iloc[i].tag1 <= 10: 
        moral_sentiments[int(MFD.iloc[i].tag1-1)] += 1
      if MFD.iloc[i].tag2 <= 10:
        moral_sentiments[int(MFD.iloc[i].tag2-1)] += 1
      if MFD.iloc[i].tag3 <= 10:
        moral_sentiments[int(MFD.iloc[i].tag3-1)] += 1
  if max(moral_sentiments) > 0:
    moral_sentiments_total[moral_sentiments.index(max(moral_sentiments))] += 1
  all_sentiments[5] += int(moral_sentiments_total[0] - moral_sentiments_total[1])
  all_sentiments[6] += int(moral_sentiments_total[2] - moral_sentiments_total[3])
  all_sentiments[7] += int(moral_sentiments_total[4] - moral_sentiments_total[5])
  all_sentiments[8] += int(moral_sentiments_total[6] - moral_sentiments_total[7])
  all_sentiments[9] += int(moral_sentiments_total[8] - moral_sentiments_total[9])
  return all_sentiments

In [171]:
def get_emotion(all_sentiments, post):
  x = te.get_emotion(str(post))
  all_sentiments[0] += int(x["Happy"])
  all_sentiments[1] += int(x["Angry"])
  all_sentiments[2] += int(x["Surprise"])
  all_sentiments[3] += int(x["Sad"])
  all_sentiments[4] += int(x["Fear"])
  return all_sentiments

In [172]:
import text2emotion as te
import pandas as pd

# To get arrays of all sentiments 
def get_sentiments(body, MFD):
  sentiments = [[],[],[],[],[],[],[],[],[],[]] # ["Happy", "Angry", "Surpr", "Sad", "Fear"], ["Harm", "Fairness", "Ingroup", "Authority", "Purity"]
  sentiment_score = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  for post in body:
    get_emotion(sentiment_score, post)
    get_morality(sentiment_score, post, MFD)
    for i in range(10):
      sentiments[i].append(sentiment_score[i])
  return sentiments

In [182]:
emotions = ["Happy", "Angry", "Surpr", "Sad", "Fear"]
moral_sentiments = ["Harm", "Fairness", "Ingroup", "Authority", "Purity"]

def get_correlation(body):
  print("Pearson")
  ans = get_sentiments(body, MFD)
  print("     ", end = "  ")
  for x in moral_sentiments: 
    print(x, end ="|")
  print()
  for i in range(5):
    print(emotions[i], end =": ")
    for j in range(5, 10):
      print(pcorr(ans[i], ans[j]), end = "|")
    print()
  print()
  print("Spearman")
  print("     ", end = "  ")
  for x in moral_sentiments: 
    print(x, end ="|")
  print()
  for i in range(5):
    print(emotions[i], end =": ")
    for j in range(5, 10):
      print(scorr(ans[i], ans[j]), end = "|")
    print()

## Results

In [184]:
get_correlation(posts)

Pearson
       Harm|Fairness|Ingroup|Authority|Purity|
Happy: -0.9927727131284692|0.8868223562587987|0.9947622924659225|0.9923317463073544|-0.9983154961647555|
Angry: -0.9950178092992369|0.885153465955621|0.997175876505794|0.9870778876113391|-0.9945738118731234|
Surpr: -0.9984699656869379|0.8658715635047822|0.9968022146161513|0.9874336824338854|-0.9922982691946791|
Sad: -0.9948631713202529|0.8921753077390074|0.9972849028513078|0.9938384544906319|-0.9987170408144228|
Fear: -0.9967289265473858|0.89097778353529|0.9992969468177181|0.9937774689722189|-0.9974374557118122|

Spearman
       Harm|Fairness|Ingroup|Authority|Purity|
Happy: -0.9996393278869923|0.9220331047799168|0.9997974932072445|0.9995376392395828|-0.9998204455904374|
Angry: -0.9993325183263022|0.9210059107760309|0.9995527371031669|0.9991781843940696|-0.9995751696190754|
Surpr: -0.9997149703274669|0.9222964211550375|0.9998880983053582|0.9996111400674251|-0.999910045671549|
Sad: -0.9997109938186577|0.9223166514649708|0.9998887666

In [185]:
get_correlation(comments)

Pearson
       Harm|Fairness|Ingroup|Authority|Purity|
Happy: -0.9987497133019937|0.996598349184149|0.999060196436882|0.9986233480792734|-0.9983217816425772|
Angry: -0.9977802439284092|0.997379476474602|0.9986351828668839|0.9992231959380862|-0.9992467628205547|
Surpr: -0.9989836019761726|0.9963309078932534|0.9987025067342825|0.9983850031467945|-0.9982921745413224|
Sad: -0.9979116142272607|0.9977278595717787|0.9991965807146531|0.9996170292055975|-0.9994863579930771|
Fear: -0.9990015373009999|0.9965332010496843|0.9988545719422998|0.9986044862059136|-0.9983871155028285|

Spearman
       Harm|Fairness|Ingroup|Authority|Purity|
Happy: -0.9999837813137111|0.9999640277510267|0.9999994801184084|0.999998593543703|-0.9999954593311967|
Angry: -0.9999788135393612|0.9999594388219519|0.9999953267097152|0.9999943919685933|-0.9999915501244377|
Surpr: -0.9999838442396447|0.9999642830835578|0.999999779086084|0.9999988653798079|-0.9999957889655872|
Sad: -0.9999838208427987|0.9999643588543848|0.9999997738